<a href="https://colab.research.google.com/github/neuroidss/silent_speech/blob/main/silent_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [1]:
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/silent_speech"):
    %cd "/content/drive/MyDrive/silent_speech"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/dgaddy/silent_speech.git
    %cd "/content/drive/MyDrive/silent_speech"
    !wget https://zenodo.org/record/4064409/files/emg_data.tar.gz?download=1 -O emg_data.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/emg_data.tar.gz -C /content
    !ln -s /content/emg_data ./emg_data
    !wget https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz
    !tar -xzf /content/drive/MyDrive/silent_speech/text_alignments.tar.gz -C /content
    !ln -s /content/text_alignments ./text_alignments
    !git clone https://github.com/NVIDIA/nv-wavenet.git nv_wavenet
    %cd /content/silent_speech/nv_wavenet/pytorch
    !make
    !python build.py install
    %cd /content/silent_speech
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.pbmm
    !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.7.0/deepspeech-0.7.0-models.scorer




/content/drive/MyDrive
Cloning into 'silent_speech'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 17), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (43/43), done.
/content/drive/MyDrive/silent_speech
--2022-02-03 23:15:50--  https://zenodo.org/record/4064409/files/emg_data.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3919507637 (3.6G) [application/octet-stream]
Saving to: ‘emg_data.tar.gz’

emg_data.tar.gz      16%[==>                 ] 631.40M  7.12MB/s    eta 6m 34s 

# Environment Setup

In [4]:
!pip install pysndfile
#==1.0.28

In [ ]:
!pip install absl-py librosa soundfile matplotlib scipy numba jiwer unidecode deepspeech==0.8.2 praat-textgrids

# Running

In [ ]:
!python wavenet_model.py --output_directory "./models/wavenet_model/" --silent_data_directories ""


In [ ]:

#!python transduction_model.py --data_size_fraction 0.5 --batch_size 1 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 2 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"
#!python transduction_model.py --batch_size 32 --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"



In [ ]:
!python transduction_model.py --pretrained_wavenet_model "./models/wavenet_model/wavenet_model.pt" --output_directory "./models/transduction_model/"


In [ ]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output


In [ ]:
!python evaluate.py --models ./models/transduction_model/model.pt --pretrained_wavenet_model ./models/wavenet_model/wavenet_model.pt --output_directory evaluation_output --testset_file testset_origdev.json
